# PERSUADE 2.0 Dataset Notebook
## 2. Counterfactuals

Tests related to the PERSUADE 2.0 essay dataset.

### Load dataset

In [1]:
import pandas as pd
# load dataset
df = pd.read_csv('tagged_dataset.csv')
df

,essay_id_comp,full_text,holistic_essay_score,word_count,prompt_name,stance,formality,sentiment
0,423A1CA112E2,phones\r\n\r\nmodern humans today are always o...,3,378,Phones and driving,2.0,0,0.0
1,BC75783F96E3,this essay will explain if drivers should or s...,4,432,Phones and driving,1.0,0,0.0
2,74C8BC7417DE,driving while the use of cellular devices\r\n\...,2,179,Phones and driving,1.0,0,1.0
3,A8445CABFECE,phones & driving\r\n\r\ndrivers should not be ...,3,221,Phones and driving,1.0,1,1.0
4,6B4F7A0165B9,cell phone operation while driving\r\n\r\nthe ...,4,334,Phones and driving,1.0,0,1.0
...,...,...,...,...,...,...,...,...
25073,AFEC37C2D43F,there has been at least one point in everyone'...,5,547,Seeking multiple opinions,2.0,0,1.0
25074,D46BCB48440A,"when people ask for advice,they sometimes talk...",4,373,Seeking multiple opinions,0.0,1,0.0
25075,0FB0700DAF44,"during a group project, have you ever asked a ...",4,631,Seeking multiple opinions,0.0,0,0.0
25076,D72CB1C11673,making choices in life can be very difficult. ...,4,417,Seeking multiple opinions,0.0,0,0.0


### Classifiers

In [1]:
import requests

# convert results list to numeric labels
stance_mapping = {
    'PRO': 0,
    'CON': 1,
    'NEUTRAL': 2
}

# calls local Ollama API Mistral instance with a defined text prompt
def query_ollama(prompt, model, temperature=0):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": model,
              "prompt": prompt,
              "stream": False,
              "options": {
                  "temperature": temperature
              }
        }
    )
    return response.json()["response"].strip()

# builds stance prompt
def get_stance_prompt(essay):
    return (
        "Stance classification is the task of determining the expressed or implied opinion, or stance, of a statement"
        " toward a certain, specified target. The following statements are social media posts expressing opinions about entities.\n"
        "Each statement can either be 'PRO' or 'CON' toward their associated entity.\n"
        "entity: Atheism\n"
        "statement: Leaving Christianity enables you to love the people you once rejected. #freethinker #Christianity #SemST\n"
        "stance: PRO\n"
        "entity: Feminist Movement\n"
        "statement: Always a delight to see chest-drumming alpha males hiss and scuttle backwards up the wall when a feminist enters the room. #manly #SemST\n"
        "stance: PRO\n"
        "entity: Christianity\n"
        "statement: AlharbiF I’ll bomb anything I can get my hands on, especially if THEY aren’t christian. #graham2016 #GOP #SemST\n"
        "stance: CON\n"
        "entity: Hillary Clinton\n"
        "statement: Would you wanna be in a long term relationship with some bitch that hides her emails, & lies to your face? Then #Dontvote #SemST\n"
        "stance: CON\n"
        "Analyze the following statement and determine its stance towards the entity.\n"
        "Respond with a single word: 'PRO' or 'CON'. Only return the stance as a single word, and no other text.'\n"
        f"statement:\n{essay}\n"
        "stance:"
    )

# standalone stance classifier
def stance_classifier(text):
    prompt = get_stance_prompt(text)
    response = query_ollama(prompt, model="mistral")

    response_upper = response.upper()
    if 'CON' in response_upper:
        return stance_mapping['CON']
    elif 'PRO' in response_upper:
        return stance_mapping['PRO']
    elif 'NEUTRAL' in response_upper:
        return stance_mapping['NEUTRAL']
    else:
        return None  # unclear

/Users/alejandro/Library/Mobile Documents/com~apple~CloudDocs/Exeter/Season 3/StyleOverSubstance/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
#from transformers import AutoModelForSequenceClassification, AutoTokenizer

# load tokenizer and model weights
model_name = 's-nlp/deberta-large-formality-ranker'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

id2formality = {0: "formal", 1: "informal"}

# standalone formality classifier
def formality_classifier(text, threshold=0.50):
    # tokenization
    encoding = tokenizer(
        text,
        add_special_tokens=True,
        return_token_type_ids=True,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )

    # get output
    output = model(**encoding)
    scores = output.logits.softmax(dim=1)[0]  # get first (and only) row

    # get score
    formality_score = {id2formality[idx]: score.item() for idx, score in enumerate(scores)}

    # apply a threshold
    if formality_score['formal'] > threshold:
        return 0  # formal
    elif formality_score['informal'] > threshold:
        return 1  # informal
    else:
        return None  # uncertain

In [5]:
from transformers import pipeline
sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")

# standalone sentiment classifier
def sentiment_classifier(text, threshold=0.90):
    # run classifier
    result = sentiment_analysis(text, truncation=True, max_length=512)[0]

    label = result['label']
    score = result['score']

    # apply a threshold
    if label == 'POSITIVE' and score > threshold:
        return 0  # positive
    elif label == 'NEGATIVE' and score > threshold:
        return 1  # negative
    else:
        return None  # uncertain

Device set to use cpu


### Generation

In [7]:
# dictionaries to be used in the prompt

RUBRIC = """
- SCORE OF 6: An essay in this category demonstrates clear and consistent mastery, although it may have a few minor errors. A typical essay effectively and insightfully develops a point of view on the issue and demonstrates outstanding critical thinking, using clearly appropriate examples, reasons, and other evidence to support its position; the essay is well organized and clearly focused, demonstrating clear coherence and smooth progression of ideas; the essay exhibits skillful use of language, using a varied, accurate, and apt vocabulary and demonstrates meaningful variety in sentence structure; the essay is free of most errors in grammar, usage, and mechanics.
- SCORE OF 5: An essay in this category demonstrates reasonably consistent mastery, although it will have occasional errors or lapses in quality. A typical essay effectively develops a point of view on the issue and demonstrates strong critical thinking, generally using appropriate examples, reasons, and other evidence to support its position; the essay is well organized and focused, demonstrating coherence and progression of ideas; the essay exhibits facility in the use of language, using appropriate vocabulary demonstrates variety in sentence structure; the essay is generally free of most errors in grammar, usage, and mechanics.
- SCORE OF 4: An essay in this category demonstrates adequate mastery, although it will have lapses in quality. A typical essay develops a point of view on the issue and demonstrates competent critical thinking, using adequate examples, reasons, and other evidence to support its position; the essay is generally organized and focused, demonstrating some coherence and progression of ideas exhibits adequate; the essay may demonstrate inconsistent facility in the use of language, using generally appropriate vocabulary demonstrates some variety in sentence structure; the essay may have some errors in grammar, usage, and mechanics.
- SCORE OF 3: An essay in this category demonstrates developing mastery, and is marked by ONE OR MORE of the following weaknesses: develops a point of view on the issue, demonstrating some critical thinking, but may do so inconsistently or use inadequate examples, reasons, or other evidence to support its position; the essay is limited in its organization or focus, or may demonstrate some lapses in coherence or progression of ideas displays; the essay may demonstrate facility in the use of language, but sometimes uses weak vocabulary or inappropriate word choice and/or lacks variety or demonstrates problems in sentence structure; the essay may contain an accumulation of errors in grammar, usage, and mechanics.
- SCORE OF 2: An essay in this category demonstrates little mastery, and is flawed by ONE OR MORE of the following weaknesses: develops a point of view on the issue that is vague or seriously limited, and demonstrates weak critical thinking, providing inappropriate or insufficient examples, reasons, or other evidence to support its position; the essay is poorly organized and/or focused, or demonstrates serious problems with coherence or progression of ideas; the essay displays very little facility in the use of language, using very limited vocabulary or incorrect word choice and/or demonstrates frequent problems in sentence structure; the essay contains errors in grammar, usage, and mechanics so serious that meaning is somewhat obscured.
- SCORE OF 1: An essay in this category demonstrates very little or no mastery, and is severely flawed by ONE OR MORE of the following weaknesses: develops no viable point of view on the issue, or provides little or no evidence to support its position; the essay is disorganized or unfocused, resulting in a disjointed or incoherent essay; the essay displays fundamental errors in vocabulary and/or demonstrates severe flaws in sentence structure; the essay contains pervasive errors in grammar, usage, or mechanics that persistently interfere with meaning.
"""

stance_opposite = {
    'PRO': 'CON',
    'CON': 'PRO',
}

formality_opposite = {
    'FORMAL': 'INFORMAL',
    'INFORMAL': 'FORMAL'
}

sentiment_opposite = {
    'POSITIVE': 'NEGATIVE',
    'NEGATIVE': 'POSITIVE'
}

attribute_context = {
    'stance': (
        "In a PRO stance, the author clearly supports the topic by providing strong reasons in favor of it, emphasizing benefits, positives, and supportive arguments.\n"
        "In a CON stance, the author clearly opposes the topic, focusing on drawbacks, risks, or negative consequences.\n"
        "Switching stance requires presenting the OPPOSITE arguments.\n"
    ),
    'formality': (
        "FORMAL writing is structured and objective. It uses academic language, avoids contractions (like 'don't' or 'can't'), and avoids casual expressions.\n"
        "It favors complex sentences, third-person point of view, and professional tone.\n"
        "INFORMAL writing is conversational, uses contractions and casual language, favors first-person or second-person, and may include slang or simple sentence structures.\n"
        "To change from FORMAL to INFORMAL, make the writing sound more like spoken language. Use really simple, repeated words and structures, expressions. Use contractions and slang terms. \n"
        "To change from INFORMAL to FORMAL, eliminate contractions, use more precise vocabulary, and avoid casual phrasing.\n"
    ),
    'sentiment': (
        "POSITIVE sentiment expresses approval, optimism, or positive feelings.\n"
        "NEGATIVE sentiment expresses disapproval, criticism, or negative feelings.\n"
    )
}

# function to build the counterfactual prompt to be fed to the LLM
def get_counterfactual_prompt(text, attribute, original_label, og_score):
    if attribute == 'stance':
        opposite_label = stance_opposite.get(original_label, "UNKNOWN")
        instruction = f"the current stance is {original_label}. Change it to {opposite_label}"
    elif attribute == 'formality':
        opposite_label = formality_opposite.get(original_label, "UNKNOWN")
        instruction = f"the current style is '{original_label}'. Change it to {opposite_label}."
    elif attribute == 'sentiment':
        opposite_label = sentiment_opposite.get(original_label, "UNKNOWN")
        instruction = f"the current sentiment is '{original_label}'. Change it to {opposite_label}."
    else:
        instruction = "Unknown attribute."

    return (
        "You are a skilled editor. Your task is to rewrite the following essay carefully."
        f"Keep the structure, arguments, and topic the same, but {instruction}. "
        f"Context:\n{attribute_context[attribute]}\n\n"
        "Do not add new ideas or remove key points. Keep the length and organization similar.\n"
        "Keep the same level of language proficiency from the original essay, including misspellings, punctuation and capitalization.\n"
        f"This essay has a score of {og_score} points on a 6-point scale. Your rewriting must meet the exact criteria as the rubric details for a score of {og_score}. The marking rubric is defined as follows:\n{RUBRIC}\n\n"
        "Respond with the rewritten essay only.\n\n"
        f"Essay:\n{text}"
    )

# generates a counterfactual given text, attribute, original label and the essay score
def generate_counterfactual(text, attribute, original_label, og_score):
    prompt = get_counterfactual_prompt(text, attribute, original_label, og_score)
    response = query_ollama(prompt, model="gemma3:12b", temperature=0.7)
    return response

In [15]:
import random

# generates x amount of counterfactuals and validates them using the given attribute classifier.
# essays are discarded if they do not align with correct classification.
# uses stratified sampling by holistic_essay_score attribute
def collect_validated_flips(
    df,
    attribute: str,                   # 'stance', 'formality', or 'sentiment'
    from_class: int,                  # e.g., 0 (for PRO)
    classifier_fn=None,               # e.g., stance_classifier
    label_mapping=None,               # e.g., {'PRO': 0, 'CON': 1, 'NEUTRAL': 2}
    prefix="ST",                      # e.g., 'ST'
    target_count=100
):

    # reverse the label mapping
    reverse_label_mapping = {v: k for k, v in label_mapping.items()}

    from_label = reverse_label_mapping[from_class]
    to_label = {
        'stance': stance_opposite,
        'formality': formality_opposite,
        'sentiment': sentiment_opposite
    }[attribute][from_label]
    to_class = label_mapping[to_label]

    successful_flips = []

    # stratified ssampling by score
    score_bins = {score: group.index.tolist() for score, group in df.groupby('holistic_essay_score')}
    bin_targets = {score: max(1, int(len(indices) / len(df) * target_count)) for score, indices in score_bins.items()}

    total_flips = 0 # count total flips
    total_attempts = 1
    for score, indices in score_bins.items():
        if total_flips >= target_count:
            print(f"Done. Total flips is {total_flips} out of {target_count}")
            break

        random.shuffle(indices)
        sample_count = 0

        for idx in indices:
            if sample_count >= bin_targets[score]:
                break

            row = df.loc[idx]
            if row[attribute] != from_class:
                continue

            original_text = row['full_text']
            essay_code_original = f"{idx}-O"
            og_score = row['holistic_essay_score']

            # generate counterfactual
            print(f"----- Attempt: {total_attempts} -----")
            try:
                print(f"Generating counterfactual... Attribute: {attribute}. {from_label} -> {to_label}")
                flipped_text = generate_counterfactual(original_text, attribute, from_label, og_score)
                print("Counterfactual generated.")
                new_label = classifier_fn(flipped_text)
                print(f"New label: {reverse_label_mapping[new_label]}")
            except Exception as e:
                print(f"[!] Error at idx {idx}: {e}")
                continue

            print(f"Flip Attempt: {from_label} → {to_label} | Classified as: {new_label}")

            if new_label == to_class:
                print("✓ - Correctly generated!")
                essay_code_cf = f"{idx}-C{prefix}"

                successful_flips.append({
                    "essay_code": essay_code_original,
                    "full_text": original_text,
                    "attribute": attribute,
                    "label": from_class,
                    "score_og": row['holistic_essay_score']
                })

                successful_flips.append({
                    "essay_code": essay_code_cf,
                    "full_text": flipped_text,
                    "attribute": attribute,
                    "label": to_class,
                    "score_og": row['holistic_essay_score']
                })

                sample_count += 1
                total_flips += 1
            else:
                print("X - Incorrectly generated!")
            total_attempts += 1

            if len(successful_flips) >= target_count * 2:
                break

    print("---------------------------------------")
    print(f"Generated {total_flips} counterfactuals with {total_attempts - 1} attempts.")

    # second pass to fill counterfactuals if target not reached
    if total_flips < target_count:
        print(f"\nSecond pass: filling {target_count - total_flips} missing flips...")

        remaining_indices = df[df[attribute] == from_class].index.tolist()
        random.shuffle(remaining_indices)

        for idx in remaining_indices:
            if total_flips >= target_count:
                break

            row = df.loc[idx]
            original_text = row['full_text']
            essay_code_original = f"{idx}-O"
            og_score = row['holistic_essay_score']

            print(f"----- Attempt: {total_attempts} -----")
            try:
                print(f"Generating counterfactual... Attribute: {attribute}. {from_label} -> {to_label}")
                flipped_text = generate_counterfactual(original_text, attribute, from_label, og_score)
                print("Counterfactual generated.")
                new_label = classifier_fn(flipped_text)
                print(f"New label: {reverse_label_mapping[new_label]}")
            except Exception as e:
                print(f"[!] Error at idx {idx}: {e}")
                continue

            print(f"Flip Attempt: {from_label} → {to_label} | Classified as: {new_label}")

            if new_label == to_class:
                print("✓ - Correctly generated!")
                essay_code_cf = f"{idx}-C{prefix}"

                successful_flips.append({
                    "essay_code": essay_code_original,
                    "full_text": original_text,
                    "attribute": attribute,
                    "label": from_class,
                    "score_og": row['holistic_essay_score']
                })

                successful_flips.append({
                    "essay_code": essay_code_cf,
                    "full_text": flipped_text,
                    "attribute": attribute,
                    "label": to_class,
                    "score_og": row['holistic_essay_score']
                })

                total_flips += 1
            else:
                print("X - Incorrectly generated!")
            total_attempts += 1

    print(f"Generated {total_flips} counterfactuals")

    return pd.DataFrame(successful_flips)

In [16]:
# generate PRO -> CON flipped stance counterfactuals

stance_pro_to_con_df = collect_validated_flips(
    df=df,
    attribute="stance",
    from_class=0,  # PRO
    classifier_fn=stance_classifier,
    label_mapping={"PRO": 0, "CON": 1, "NEUTRAL": 2},
    prefix="ST",
    target_count=100
)

stance_pro_to_con_df

----- Attempt: 1 -----
Generating counterfactual... Attribute: stance. PRO -> CON
Counterfactual generated.
New label: CON
Flip Attempt: PRO → CON | Classified as: 1
✓ - Correctly generated!
----- Attempt: 2 -----
Generating counterfactual... Attribute: stance. PRO -> CON
Counterfactual generated.
New label: CON
Flip Attempt: PRO → CON | Classified as: 1
✓ - Correctly generated!
----- Attempt: 3 -----
Generating counterfactual... Attribute: stance. PRO -> CON
Counterfactual generated.
New label: CON
Flip Attempt: PRO → CON | Classified as: 1
✓ - Correctly generated!
----- Attempt: 4 -----
Generating counterfactual... Attribute: stance. PRO -> CON
Counterfactual generated.
New label: CON
Flip Attempt: PRO → CON | Classified as: 1
✓ - Correctly generated!
----- Attempt: 5 -----
Generating counterfactual... Attribute: stance. PRO -> CON
Counterfactual generated.
New label: CON
Flip Attempt: PRO → CON | Classified as: 1
✓ - Correctly generated!
----- Attempt: 6 -----
Generating counterfact

,essay_code,full_text,attribute,label,score_og
0,15984-O,i would try to do better in school so i would ...,stance,0,1
1,15984-CST,i would try to do worse in school so i would n...,stance,1,1
2,1972-O,cars have been producing smog and causing prob...,stance,0,1
3,1972-CST,cars have been producing smog and causing prob...,stance,1,1
4,10486-O,im going to tell you if the facial action codi...,stance,0,1
...,...,...,...,...,...
195,2854-CST,"do you think you could live without your car, ...",stance,1,3
196,6896-O,at every school there are extracurricular acti...,stance,0,4
197,6896-CST,at every school there are extracurricular acti...,stance,1,4
198,14704-O,"dear principal,\r\n\r\ni have heard around tha...",stance,0,4


In [17]:
# generate CON -> PRO flipped stance counterfactuals

stance_con_to_pro_df = collect_validated_flips(
    df=df,
    attribute="stance",
    from_class=1,  # CON
    classifier_fn=stance_classifier,
    label_mapping={"PRO": 0, "CON": 1, "NEUTRAL": 2},
    prefix="ST",
    target_count=100
)

stance_con_to_pro_df

----- Attempt: 1 -----
Generating counterfactual... Attribute: stance. CON -> PRO
Counterfactual generated.
New label: PRO
Flip Attempt: CON → PRO | Classified as: 0
✓ - Correctly generated!
----- Attempt: 2 -----
Generating counterfactual... Attribute: stance. CON -> PRO
Counterfactual generated.
New label: PRO
Flip Attempt: CON → PRO | Classified as: 0
✓ - Correctly generated!
----- Attempt: 3 -----
Generating counterfactual... Attribute: stance. CON -> PRO
Counterfactual generated.
New label: PRO
Flip Attempt: CON → PRO | Classified as: 0
✓ - Correctly generated!
----- Attempt: 4 -----
Generating counterfactual... Attribute: stance. CON -> PRO
Counterfactual generated.
New label: PRO
Flip Attempt: CON → PRO | Classified as: 0
✓ - Correctly generated!
----- Attempt: 5 -----
Generating counterfactual... Attribute: stance. CON -> PRO
Counterfactual generated.
New label: CON
Flip Attempt: CON → PRO | Classified as: 1
X - Incorrectly generated!
----- Attempt: 6 -----
Generating counterfa

,essay_code,full_text,attribute,label,score_og
0,8603-O,studying venus is something that you would hav...,stance,1,1
1,8603-CST,studying venus is something that you absolutel...,stance,0,1
2,18824-O,the electoral college is a way of voting for p...,stance,1,1
3,18824-CST,the electoral college is a way of voting for p...,stance,0,1
4,19381-O,i think we should keep the electoral collage b...,stance,1,1
...,...,...,...,...,...
195,3623-CST,summer projects are a great way to keep studen...,stance,0,5
196,15235-O,"dear principal,\r\n\r\ni believe that having a...",stance,1,4
197,15235-CST,"dear principal,\n\ni believe that having a b a...",stance,0,4
198,994-O,phones and driving\r\n\r\nalthough cell phones...,stance,1,3


In [18]:
# generate POSITIVE -> NEGATIVE flipped sentiment counterfactuals

sentiment_positive_to_negative_df = collect_validated_flips(
    df=df,
    attribute="sentiment",
    from_class=0,  # POSITIVE
    classifier_fn=sentiment_classifier,
    label_mapping={"POSITIVE": 0, "NEGATIVE": 1},
    prefix="SE",
    target_count=100
)

sentiment_positive_to_negative_df

----- Attempt: 1 -----
Generating counterfactual... Attribute: sentiment. POSITIVE -> NEGATIVE
Counterfactual generated.
New label: NEGATIVE
Flip Attempt: POSITIVE → NEGATIVE | Classified as: 1
✓ - Correctly generated!
----- Attempt: 2 -----
Generating counterfactual... Attribute: sentiment. POSITIVE -> NEGATIVE
Counterfactual generated.
New label: POSITIVE
Flip Attempt: POSITIVE → NEGATIVE | Classified as: 0
X - Incorrectly generated!
----- Attempt: 3 -----
Generating counterfactual... Attribute: sentiment. POSITIVE -> NEGATIVE
Counterfactual generated.
New label: NEGATIVE
Flip Attempt: POSITIVE → NEGATIVE | Classified as: 1
✓ - Correctly generated!
----- Attempt: 4 -----
Generating counterfactual... Attribute: sentiment. POSITIVE -> NEGATIVE
Counterfactual generated.
New label: NEGATIVE
Flip Attempt: POSITIVE → NEGATIVE | Classified as: 1
✓ - Correctly generated!
----- Attempt: 5 -----
Generating counterfactual... Attribute: sentiment. POSITIVE -> NEGATIVE
Counterfactual generated.
N

,essay_code,full_text,attribute,label,score_og
0,5756-O,luke bomberger was just a normal high school g...,sentiment,0,1
1,5756-CSE,luke bomberger was just a normal high school g...,sentiment,1,1
2,8842-O,because he said the sun is more colse from us ...,sentiment,0,1
3,8842-CSE,because he insisted the sun was more colse fro...,sentiment,1,1
4,11281-O,i'm fore the value of using this technology to...,sentiment,0,1
...,...,...,...,...,...
195,5892-CSE,luke is decidedly not courageous because he jo...,sentiment,1,1
196,16245-O,i personally believe that driverless cars are ...,sentiment,0,3
197,16245-CSE,i personally believe that driverless cars are ...,sentiment,1,3
198,16939-O,the further development of driverless cars sho...,sentiment,0,5


In [19]:
# generate NEGATIVE -> POSITIVE flipped sentiment counterfactuals

sentiment_negative_to_positive_df = collect_validated_flips(
    df=df,
    attribute="sentiment",
    from_class=1,  # NEGATIVE
    classifier_fn=sentiment_classifier,
    label_mapping={"POSITIVE": 0, "NEGATIVE": 1},
    prefix="SE",
    target_count=100
)

sentiment_negative_to_positive_df

----- Attempt: 1 -----
Generating counterfactual... Attribute: sentiment. NEGATIVE -> POSITIVE
Counterfactual generated.
New label: POSITIVE
Flip Attempt: NEGATIVE → POSITIVE | Classified as: 0
✓ - Correctly generated!
----- Attempt: 2 -----
Generating counterfactual... Attribute: sentiment. NEGATIVE -> POSITIVE
Counterfactual generated.
New label: POSITIVE
Flip Attempt: NEGATIVE → POSITIVE | Classified as: 0
✓ - Correctly generated!
----- Attempt: 3 -----
Generating counterfactual... Attribute: sentiment. NEGATIVE -> POSITIVE
Counterfactual generated.
New label: POSITIVE
Flip Attempt: NEGATIVE → POSITIVE | Classified as: 0
✓ - Correctly generated!
----- Attempt: 4 -----
Generating counterfactual... Attribute: sentiment. NEGATIVE -> POSITIVE
Counterfactual generated.
New label: POSITIVE
Flip Attempt: NEGATIVE → POSITIVE | Classified as: 0
✓ - Correctly generated!
----- Attempt: 5 -----
Generating counterfactual... Attribute: sentiment. NEGATIVE -> POSITIVE
Counterfactual generated.
New

,essay_code,full_text,attribute,label,score_og
0,1451-O,no one is evan going to read these things so t...,sentiment,1,1
1,1451-CSE,no one is evan going to read these things so t...,sentiment,0,1
2,23542-O,do you think student would benefit from being ...,sentiment,1,1
3,23542-CSE,do you think student would benefit from being ...,sentiment,0,1
4,8311-O,-\r\n\r\nvenus is an second planet for the sun...,sentiment,1,1
...,...,...,...,...,...
195,17151-CSE,driverless cars are coming to the roads and th...,sentiment,0,2
196,7663-O,"the article ""the challenge of exploring venus""...",sentiment,1,4
197,7663-CSE,"the article ""the challenge of exploring venus""...",sentiment,0,4
198,6606-O,have you ever done something you didn't want t...,sentiment,1,4


In [20]:
# generate FORMAL -> INFORMAL flipped formality counterfactuals

formality_formal_to_informal_df = collect_validated_flips(
    df=df,
    attribute="formality",
    from_class=0,  # FORMAL
    classifier_fn=formality_classifier,
    label_mapping={"FORMAL": 0, "INFORMAL": 1},
    prefix="FO",
    target_count=100
)

formality_formal_to_informal_df

----- Attempt: 1 -----
Generating counterfactual... Attribute: formality. FORMAL -> INFORMAL
Counterfactual generated.
New label: FORMAL
Flip Attempt: FORMAL → INFORMAL | Classified as: 0
X - Incorrectly generated!
----- Attempt: 2 -----
Generating counterfactual... Attribute: formality. FORMAL -> INFORMAL
Counterfactual generated.
New label: FORMAL
Flip Attempt: FORMAL → INFORMAL | Classified as: 0
X - Incorrectly generated!
----- Attempt: 3 -----
Generating counterfactual... Attribute: formality. FORMAL -> INFORMAL
Counterfactual generated.
New label: INFORMAL
Flip Attempt: FORMAL → INFORMAL | Classified as: 1
✓ - Correctly generated!
----- Attempt: 4 -----
Generating counterfactual... Attribute: formality. FORMAL -> INFORMAL
Counterfactual generated.
New label: INFORMAL
Flip Attempt: FORMAL → INFORMAL | Classified as: 1
✓ - Correctly generated!
----- Attempt: 5 -----
Generating counterfactual... Attribute: formality. FORMAL -> INFORMAL
Counterfactual generated.
New label: INFORMAL
F

,essay_code,full_text,attribute,label,score_og
0,1278-O,the selctions i just read contain one common s...,formality,0,1
1,1278-CFO,the stuff i just read all talks about one thin...,formality,1,1
2,12401-O,many people might think that the face on mars ...,formality,0,1
3,12401-CFO,a lot of folks think the face on Mars was made...,formality,1,1
4,9316-O,"in ""the challenge of exploring venus,"" the aut...",formality,0,1
...,...,...,...,...,...
195,4896-CFO,"in the story ""a cowboy who rode the waves"" it ...",formality,1,4
196,24883-O,why is it impirtant to have multiple people gi...,formality,0,3
197,24883-CFO,why is it impirtant to have multiple people gi...,formality,1,3
198,22572-O,do you think that students would benefit from ...,formality,0,3


In [21]:
# generate INFORMAL -> FORMAL flipped formality counterfactuals

formality_informal_to_formal_df = collect_validated_flips(
    df=df,
    attribute="formality",
    from_class=1,  # INFORMAL
    classifier_fn=formality_classifier,
    label_mapping={"FORMAL": 0, "INFORMAL": 1},
    prefix="FO",
    target_count=100
)

formality_informal_to_formal_df

----- Attempt: 1 -----
Generating counterfactual... Attribute: formality. INFORMAL -> FORMAL
Counterfactual generated.
New label: FORMAL
Flip Attempt: INFORMAL → FORMAL | Classified as: 0
✓ - Correctly generated!
----- Attempt: 2 -----
Generating counterfactual... Attribute: formality. INFORMAL -> FORMAL
Counterfactual generated.
New label: FORMAL
Flip Attempt: INFORMAL → FORMAL | Classified as: 0
✓ - Correctly generated!
----- Attempt: 3 -----
Generating counterfactual... Attribute: formality. INFORMAL -> FORMAL
Counterfactual generated.
New label: FORMAL
Flip Attempt: INFORMAL → FORMAL | Classified as: 0
✓ - Correctly generated!
----- Attempt: 4 -----
Generating counterfactual... Attribute: formality. INFORMAL -> FORMAL
Counterfactual generated.
New label: FORMAL
Flip Attempt: INFORMAL → FORMAL | Classified as: 0
✓ - Correctly generated!
----- Attempt: 5 -----
Generating counterfactual... Attribute: formality. INFORMAL -> FORMAL
Counterfactual generated.
New label: FORMAL
Flip Attemp

,essay_code,full_text,attribute,label,score_og
0,1859-O,"the passage ""in german suburb, life goes on wi...",formality,1,1
1,1859-CFO,"The passage ""in german suburb, life goes on wi...",formality,0,1
2,19152-O,to me i think we should change the election by...,formality,1,1
3,19152-CFO,"To me, I believe that we should alter the elec...",formality,0,1
4,8542-O,the challenge of exploring venus\r\n\r\non thi...,formality,1,1
...,...,...,...,...,...
195,18967-CFO,"candidate, what do you think when I say that w...",formality,0,1
196,19652-O,"dear senator of state,\r\n\r\ni think keeping ...",formality,1,1
197,19652-CFO,"Dear Senator of State,\n\nI believe that maint...",formality,0,1
198,9618-O,"you are smile, but you only 80 percent happy, ...",formality,1,3


In [22]:
import os

# save counterfactuals to its folder
os.makedirs("counterfactuals", exist_ok=True)

stance_pro_to_con_df.to_csv("counterfactuals/stance_pro_to_con.csv", index=False)
stance_con_to_pro_df.to_csv("counterfactuals/stance_con_to_pro.csv", index=False)

sentiment_positive_to_negative_df.to_csv("counterfactuals/sentiment_positive_to_negative.csv", index=False)
sentiment_negative_to_positive_df.to_csv("counterfactuals/sentiment_negative_to_positive.csv", index=False)

formality_formal_to_informal_df.to_csv("counterfactuals/formality_formal_to_informal.csv", index=False)
formality_informal_to_formal_df.to_csv("counterfactuals/formality_informal_to_formal.csv", index=False)

### Essay evaluation

In [61]:
# load datasets

stance_pro_to_con_df = pd.read_csv('counterfactuals/stance_pro_to_con.csv')
stance_con_to_pro_df = pd.read_csv('counterfactuals/stance_con_to_pro.csv')

sentiment_positive_to_negative_df = pd.read_csv('counterfactuals/sentiment_positive_to_negative.csv')
sentiment_negative_to_positive_df = pd.read_csv('counterfactuals/sentiment_negative_to_positive.csv')

In [23]:
from sklearn.metrics import cohen_kappa_score

# strings to be used in building the evaluation prompt
RUBRIC = """
- SCORE OF 6: An essay in this category demonstrates clear and consistent mastery, although it may have a few minor errors. A typical essay effectively and insightfully develops a point of view on the issue and demonstrates outstanding critical thinking, using clearly appropriate examples, reasons, and other evidence to support its position; the essay is well organized and clearly focused, demonstrating clear coherence and smooth progression of ideas; the essay exhibits skillful use of language, using a varied, accurate, and apt vocabulary and demonstrates meaningful variety in sentence structure; the essay is free of most errors in grammar, usage, and mechanics.
- SCORE OF 5: An essay in this category demonstrates reasonably consistent mastery, although it will have occasional errors or lapses in quality. A typical essay effectively develops a point of view on the issue and demonstrates strong critical thinking, generally using appropriate examples, reasons, and other evidence to support its position; the essay is well organized and focused, demonstrating coherence and progression of ideas; the essay exhibits facility in the use of language, using appropriate vocabulary demonstrates variety in sentence structure; the essay is generally free of most errors in grammar, usage, and mechanics.
- SCORE OF 4: An essay in this category demonstrates adequate mastery, although it will have lapses in quality. A typical essay develops a point of view on the issue and demonstrates competent critical thinking, using adequate examples, reasons, and other evidence to support its position; the essay is generally organized and focused, demonstrating some coherence and progression of ideas exhibits adequate; the essay may demonstrate inconsistent facility in the use of language, using generally appropriate vocabulary demonstrates some variety in sentence structure; the essay may have some errors in grammar, usage, and mechanics.
- SCORE OF 3: An essay in this category demonstrates developing mastery, and is marked by ONE OR MORE of the following weaknesses: develops a point of view on the issue, demonstrating some critical thinking, but may do so inconsistently or use inadequate examples, reasons, or other evidence to support its position; the essay is limited in its organization or focus, or may demonstrate some lapses in coherence or progression of ideas displays; the essay may demonstrate facility in the use of language, but sometimes uses weak vocabulary or inappropriate word choice and/or lacks variety or demonstrates problems in sentence structure; the essay may contain an accumulation of errors in grammar, usage, and mechanics.
- SCORE OF 2: An essay in this category demonstrates little mastery, and is flawed by ONE OR MORE of the following weaknesses: develops a point of view on the issue that is vague or seriously limited, and demonstrates weak critical thinking, providing inappropriate or insufficient examples, reasons, or other evidence to support its position; the essay is poorly organized and/or focused, or demonstrates serious problems with coherence or progression of ideas; the essay displays very little facility in the use of language, using very limited vocabulary or incorrect word choice and/or demonstrates frequent problems in sentence structure; the essay contains errors in grammar, usage, and mechanics so serious that meaning is somewhat obscured.
- SCORE OF 1: An essay in this category demonstrates very little or no mastery, and is severely flawed by ONE OR MORE of the following weaknesses: develops no viable point of view on the issue, or provides little or no evidence to support its position; the essay is disorganized or unfocused, resulting in a disjointed or incoherent essay; the essay displays fundamental errors in vocabulary and/or demonstrates severe flaws in sentence structure; the essay contains pervasive errors in grammar, usage, or mechanics that persistently interfere with meaning.
"""

FEW_SHOT_EVAL = """
Example essay 1 of score "4":\n
"phones and driving\n\nin this world in which we live in, cell phones are a growing market as well as cars. the fact that we depend on cell phones throught the course of our day for numerous reasons.\n\ndrivers should be able to use their cell phones while driving because its easier to operate a phone in your hand than a cell phone that is not in your hand. emergencies can occur while driving and you need to report the emergerncy while driving. cell phones does not cause as big of a distrachon that other things that is done while driving.\n\na lot of people are in an uproar about driving and using a cell phone and somehmes it is overrated. trying to operate a cell phone with one hand while maintaining driving has no danger to it. a phone that is mounted on the holder that begins to ring is harder to operate answering, dialing and switching calls.\n\nmost hmes the mounhng is not secure or has defected parts which can cause more of an issue than having it to your ear. most people have older cars and cannot a? ord to upgrade and dont have the speaker ophon, which usually allows everyone surrounding your car to know you business.\n\ni cannot say it enough, emergencies happen in a ?ash. driving and you see a accident that needs emergency alenhon you need to be able to use the cell phone to call it in. things happen so quickly now and it could be your phone call that saves their life. also if a person is lost and no gps signal but calls can be made a person should be able to dial someone by hand and get direchons out of harms way. emergency is a big deal in cell phone usage.\n\ncell phones do not cause as much as a distrachon than people make out to be. as a driver a person should always be aware of the road and be able to mulh-task when using a cell phone. most hmes its other things besides a cell phone that cause a distrachon but blames the cell phone as a scapegoat. there could be test on the driver liscence test that we should take to see if were able to drive and talk on the phone instead of just banning it.\n\ntalking on a cell phone and driving a car may be a distrachon for some but not all. it should be our call on if we are focused enough to drive while talking. although some people are said to have had bad behavior while talking and driving some are very responsible. texhng and driving is di? erent from talking and driving and that should be the boundary. each person should be accountable for their achons just as speeding."\n\n
Example Essay 2 of score "3":
"phones\n\nmodern humans today are always on their phone. they are always on their phone more than 5 hours a day no stop .all they do is text back and forward and just have group chats on social media. they even do it while driving. they are some really bad consequences when stuff happens when it comes to a phone. some certain areas in the united states ban phones from class rooms just because of it.\n\nwhen people have phones, they know about certain apps that they have .apps like facebook twitter instagram and snapchat. so like if a friend moves away and you want to be in contact you can still be in contact by posting videos or text messages. people always have different ways how to communicate with a phone. phones have changed due to our generation.\n\ndriving is one of the way how to get around. people always be on their phones while doing it. which can cause serious problems. that's why there's a thing that's called no texting while driving. that's a really important thing to remember. some people still do it because they think it's stupid. no matter what they do they still have to obey it because that's the only way how did he save.\n\nsometimes on the news there is either an accident or a suicide. it might involve someone not looking where they're going or tweet that someone sent. it either injury or death. if a mysterious number says i'm going to kill you and they know where you live but you don't know the person's contact ,it makes you puzzled and make you start to freak out. which can end up really badly.\n\nphones are fine to use and it's also the best way to come over help. if you go through a problem and you can't find help you ,always have a phone there with you. even though phones are used almost every day as long as you're safe it would come into use if you get into trouble. make sure you do not be like this phone while you're in the middle of driving. the news always updated when people do something stupid around that involves their phones. the safest way is the best way to stay safe."\n\n
Example Essay 3 of score "2":\n
"should people drive with their cellphone or not?\n\nwe all love our phones now a days and basically ever since they were invented. everyone has one or wants one or the newest latest one .ever since they were invented in 1700s people have taken the full advantage of the telephone presidents and war sargents used them to win battles and etc. now the phone has evolved from tube looking phones to brick phones to flip phones to iphones and to smartphones. with all the new technology and apps like facetime snapchat3 games music social media there's almost no way anyone could ever put their phones down. but according to statistics in 2013 about 3,154 people died in accidents 424,000 were injured. in 2013, 10% of all drivers ages 15 to 19 involved in fatal accidents were reported to be distracted at the time of the crash.\n\nso the question stands do we need to drive while on your cellphone? with all the crashes maybe we should drive without our phones but with the increments in technology we mind as well just use them cars are able to call people just like phones, so if you can call people in your car and be distracted anyways when your calling whom ever. there putting almost smartphones in the car so you can most defiantly be distracted with that. the fact that they are and already are making cars able to drive by themselves is anther reason that is very distracting just as in tulsa the electrically powered car has pilot mode so it drives itself. so many people have been distracted from the road and driving because the car can drive itself even though the car can drive itself you still need to pay attention to the road.\n\ni do understand that you don't need to be looking down when driving and i do understand that phones are distracting. so you picking up a cellphone and calling someone can be distracting. also like i said in another essay i did there is a responsibility factor to firstly own a car and secondly to even own a phone. it might seem like it is not a responsibility owning a phone but it is when you own a phone you have personal information in it. if you were lose your phone and have never put a password or security code on it you can lose your information to whoever has your phone could be a hacker could be a random thieve.\n\nso really my answer is that it really depends on your responsibility level if you can't stay off your phone and it's life then you need to let someone drive but if you can be responsible while driving then i'd say you should us your phone responsibly. cause if you drive irresponsibly you should have your phone."\n
"""

# builds the evaluation prompt to be fed to the LLM evaluator
def build_eval_prompt(text):
    return (
        f"You are an essay rater specializing in the evaluation of essays written by students from 6th to 12th grade. "
        f"Read and evaluate the essay: \n\n{FEW_SHOT_EVAL}\n"
        f"Essay to score:\n{text}\n\n"
        f"Assign it a score from 1 to 6, in increments of 1, based on this rubric:\n\n{RUBRIC}\n\n"
        f"Your response should be only a numeric value representing the score you gave."
    )

# function to evaluate all essays from a df
def evaluate_essays(df, model="llama3:8b"):
    scores = []

    for idx, row in df.iterrows():
        prompt = build_eval_prompt(row['full_text'])
        score_og = row['score_og']

        try:
            print("----------------")
            print("evaluating essay...")
            response = query_ollama(prompt, model=model)
            score = int(response.strip())

            if 1 <= score <= 6:
                # score is right
                print(f"[{idx}] evaluated. score: {score}. score_og: {score_og}")
                scores.append(score)
            else:
                print(f"invalid score from LLM at idx {idx}: {response}")
                scores.append(None)
        except Exception as e:
            print(f"error scoring essay at idx {idx}: {e}")
            scores.append(None)

    df["score_llm"] = scores

    return df

# computes qwk value from df using original score and llm score
def compute_qwk(df):
    df_clean = df.dropna(subset=["score_og", "score_llm"])

    # extract originals and counterfactuals
    originals = df_clean[df_clean["essay_code"].str.endswith("-O")]
    cf = df_clean[df_clean["essay_code"].str.contains("-C")]

    qwk_original = cohen_kappa_score(originals["score_og"], originals["score_llm"], weights='quadratic')
    qwk_cf = cohen_kappa_score(cf["score_og"], cf["score_llm"], weights='quadratic')

    print(f"QWK (Originals): {qwk_original:.4f}")
    print(f"QWK (Counterfactuals): {qwk_cf:.4f}")

    return qwk_original, qwk_cf

In [24]:
# score PRO -> CON flipped stance counterfactuals and their originals

stance_pro_to_con_scored_df = evaluate_essays(stance_pro_to_con_df, 'gemma3:12b')

stance_pro_to_con_scored_df

----------------
evaluating essay...
[0] evaluated. score: 3. score_og: 1
----------------
evaluating essay...
[1] evaluated. score: 2. score_og: 1
----------------
evaluating essay...
[2] evaluated. score: 4. score_og: 1
----------------
evaluating essay...
[3] evaluated. score: 5. score_og: 1
----------------
evaluating essay...
[4] evaluated. score: 5. score_og: 1
----------------
evaluating essay...
[5] evaluated. score: 5. score_og: 1
----------------
evaluating essay...
[6] evaluated. score: 4. score_og: 1
----------------
evaluating essay...
[7] evaluated. score: 4. score_og: 1
----------------
evaluating essay...
[8] evaluated. score: 2. score_og: 2
----------------
evaluating essay...
[9] evaluated. score: 2. score_og: 2
----------------
evaluating essay...
[10] evaluated. score: 3. score_og: 2
----------------
evaluating essay...
[11] evaluated. score: 2. score_og: 2
----------------
evaluating essay...
[12] evaluated. score: 3. score_og: 2
----------------
evaluating essay..

,essay_code,full_text,attribute,label,score_og,score_llm
0,15984-O,i would try to do better in school so i would ...,stance,0,1,3
1,15984-CST,i would try to do worse in school so i would n...,stance,1,1,2
2,1972-O,cars have been producing smog and causing prob...,stance,0,1,4
3,1972-CST,cars have been producing smog and causing prob...,stance,1,1,5
4,10486-O,im going to tell you if the facial action codi...,stance,0,1,5
...,...,...,...,...,...,...
195,2854-CST,"do you think you could live without your car, ...",stance,1,3,4
196,6896-O,at every school there are extracurricular acti...,stance,0,4,5
197,6896-CST,at every school there are extracurricular acti...,stance,1,4,3
198,14704-O,"dear principal,\r\n\r\ni have heard around tha...",stance,0,4,3


In [25]:
# score CON -> PRO flipped stance counterfactuals and their originals

stance_con_to_pro_scored_df = evaluate_essays(stance_con_to_pro_df, 'gemma3:12b')

stance_con_to_pro_scored_df

----------------
evaluating essay...
[0] evaluated. score: 4. score_og: 1
----------------
evaluating essay...
[1] evaluated. score: 4. score_og: 1
----------------
evaluating essay...
[2] evaluated. score: 3. score_og: 1
----------------
evaluating essay...
[3] evaluated. score: 3. score_og: 1
----------------
evaluating essay...
[4] evaluated. score: 3. score_og: 1
----------------
evaluating essay...
[5] evaluated. score: 2. score_og: 1
----------------
evaluating essay...
[6] evaluated. score: 1. score_og: 1
----------------
evaluating essay...
[7] evaluated. score: 3. score_og: 1
----------------
evaluating essay...
[8] evaluated. score: 3. score_og: 2
----------------
evaluating essay...
[9] evaluated. score: 3. score_og: 2
----------------
evaluating essay...
[10] evaluated. score: 4. score_og: 2
----------------
evaluating essay...
[11] evaluated. score: 4. score_og: 2
----------------
evaluating essay...
[12] evaluated. score: 2. score_og: 2
----------------
evaluating essay..

,essay_code,full_text,attribute,label,score_og,score_llm
0,8603-O,studying venus is something that you would hav...,stance,1,1,4
1,8603-CST,studying venus is something that you absolutel...,stance,0,1,4
2,18824-O,the electoral college is a way of voting for p...,stance,1,1,3
3,18824-CST,the electoral college is a way of voting for p...,stance,0,1,3
4,19381-O,i think we should keep the electoral collage b...,stance,1,1,3
...,...,...,...,...,...,...
195,3623-CST,summer projects are a great way to keep studen...,stance,0,5,5
196,15235-O,"dear principal,\r\n\r\ni believe that having a...",stance,1,4,3
197,15235-CST,"dear principal,\n\ni believe that having a b a...",stance,0,4,3
198,994-O,phones and driving\r\n\r\nalthough cell phones...,stance,1,3,5


In [26]:
# score POSITIVE -> NEGATIVE flipped sentiment counterfactuals and their originals

sentiment_positive_to_negative_scored_df = evaluate_essays(sentiment_positive_to_negative_df, 'gemma3:12b')

sentiment_positive_to_negative_scored_df

----------------
evaluating essay...
[0] evaluated. score: 4. score_og: 1
----------------
evaluating essay...
[1] evaluated. score: 3. score_og: 1
----------------
evaluating essay...
[2] evaluated. score: 2. score_og: 1
----------------
evaluating essay...
[3] evaluated. score: 2. score_og: 1
----------------
evaluating essay...
[4] evaluated. score: 3. score_og: 1
----------------
evaluating essay...
[5] evaluated. score: 5. score_og: 1
----------------
evaluating essay...
[6] evaluated. score: 3. score_og: 1
----------------
evaluating essay...
[7] evaluated. score: 3. score_og: 1
----------------
evaluating essay...
[8] evaluated. score: 4. score_og: 2
----------------
evaluating essay...
[9] evaluated. score: 2. score_og: 2
----------------
evaluating essay...
[10] evaluated. score: 6. score_og: 2
----------------
evaluating essay...
[11] evaluated. score: 3. score_og: 2
----------------
evaluating essay...
[12] evaluated. score: 3. score_og: 2
----------------
evaluating essay..

,essay_code,full_text,attribute,label,score_og,score_llm
0,5756-O,luke bomberger was just a normal high school g...,sentiment,0,1,4
1,5756-CSE,luke bomberger was just a normal high school g...,sentiment,1,1,3
2,8842-O,because he said the sun is more colse from us ...,sentiment,0,1,2
3,8842-CSE,because he insisted the sun was more colse fro...,sentiment,1,1,2
4,11281-O,i'm fore the value of using this technology to...,sentiment,0,1,3
...,...,...,...,...,...,...
195,5892-CSE,luke is decidedly not courageous because he jo...,sentiment,1,1,2
196,16245-O,i personally believe that driverless cars are ...,sentiment,0,3,4
197,16245-CSE,i personally believe that driverless cars are ...,sentiment,1,3,4
198,16939-O,the further development of driverless cars sho...,sentiment,0,5,4


In [27]:
# score NEGATIVE -> POSITIVE flipped sentiment counterfactuals and their originals

sentiment_negative_to_positive_scored_df = evaluate_essays(sentiment_negative_to_positive_df, 'gemma3:12b')

sentiment_negative_to_positive_scored_df

----------------
evaluating essay...
[0] evaluated. score: 1. score_og: 1
----------------
evaluating essay...
[1] evaluated. score: 1. score_og: 1
----------------
evaluating essay...
[2] evaluated. score: 2. score_og: 1
----------------
evaluating essay...
[3] evaluated. score: 4. score_og: 1
----------------
evaluating essay...
[4] evaluated. score: 3. score_og: 1
----------------
evaluating essay...
[5] evaluated. score: 4. score_og: 1
----------------
evaluating essay...
[6] evaluated. score: 3. score_og: 1
----------------
evaluating essay...
[7] evaluated. score: 5. score_og: 1
----------------
evaluating essay...
[8] evaluated. score: 3. score_og: 2
----------------
evaluating essay...
[9] evaluated. score: 3. score_og: 2
----------------
evaluating essay...
[10] evaluated. score: 3. score_og: 2
----------------
evaluating essay...
[11] evaluated. score: 3. score_og: 2
----------------
evaluating essay...
[12] evaluated. score: 3. score_og: 2
----------------
evaluating essay..

,essay_code,full_text,attribute,label,score_og,score_llm
0,1451-O,no one is evan going to read these things so t...,sentiment,1,1,1
1,1451-CSE,no one is evan going to read these things so t...,sentiment,0,1,1
2,23542-O,do you think student would benefit from being ...,sentiment,1,1,2
3,23542-CSE,do you think student would benefit from being ...,sentiment,0,1,4
4,8311-O,-\r\n\r\nvenus is an second planet for the sun...,sentiment,1,1,3
...,...,...,...,...,...,...
195,17151-CSE,driverless cars are coming to the roads and th...,sentiment,0,2,4
196,7663-O,"the article ""the challenge of exploring venus""...",sentiment,1,4,4
197,7663-CSE,"the article ""the challenge of exploring venus""...",sentiment,0,4,5
198,6606-O,have you ever done something you didn't want t...,sentiment,1,4,4


In [28]:
# score FORMAL -> INFORMAL flipped formality counterfactuals and their originals

formality_formal_to_informal_scored_df = evaluate_essays(formality_formal_to_informal_df, 'gemma3:12b')

formality_formal_to_informal_scored_df

----------------
evaluating essay...
[0] evaluated. score: 3. score_og: 1
----------------
evaluating essay...
[1] evaluated. score: 2. score_og: 1
----------------
evaluating essay...
[2] evaluated. score: 4. score_og: 1
----------------
evaluating essay...
[3] evaluated. score: 3. score_og: 1
----------------
evaluating essay...
[4] evaluated. score: 2. score_og: 1
----------------
evaluating essay...
[5] evaluated. score: 3. score_og: 1
----------------
evaluating essay...
[6] evaluated. score: 2. score_og: 1
----------------
evaluating essay...
[7] evaluated. score: 4. score_og: 1
----------------
evaluating essay...
[8] evaluated. score: 3. score_og: 2
----------------
evaluating essay...
[9] evaluated. score: 3. score_og: 2
----------------
evaluating essay...
[10] evaluated. score: 4. score_og: 2
----------------
evaluating essay...
[11] evaluated. score: 3. score_og: 2
----------------
evaluating essay...
[12] evaluated. score: 3. score_og: 2
----------------
evaluating essay..

,essay_code,full_text,attribute,label,score_og,score_llm
0,1278-O,the selctions i just read contain one common s...,formality,0,1,3
1,1278-CFO,the stuff i just read all talks about one thin...,formality,1,1,2
2,12401-O,many people might think that the face on mars ...,formality,0,1,4
3,12401-CFO,a lot of folks think the face on Mars was made...,formality,1,1,3
4,9316-O,"in ""the challenge of exploring venus,"" the aut...",formality,0,1,2
...,...,...,...,...,...,...
195,4896-CFO,"in the story ""a cowboy who rode the waves"" it ...",formality,1,4,5
196,24883-O,why is it impirtant to have multiple people gi...,formality,0,3,3
197,24883-CFO,why is it impirtant to have multiple people gi...,formality,1,3,3
198,22572-O,do you think that students would benefit from ...,formality,0,3,3


In [29]:
# score INFORMAL -> FORMAL flipped formality counterfactuals and their originals

formality_informal_to_formal_scored_df = evaluate_essays(formality_informal_to_formal_df, 'gemma3:12b')

formality_informal_to_formal_scored_df

----------------
evaluating essay...
[0] evaluated. score: 4. score_og: 1
----------------
evaluating essay...
[1] evaluated. score: 5. score_og: 1
----------------
evaluating essay...
[2] evaluated. score: 3. score_og: 1
----------------
evaluating essay...
[3] evaluated. score: 4. score_og: 1
----------------
evaluating essay...
[4] evaluated. score: 3. score_og: 1
----------------
evaluating essay...
[5] evaluated. score: 4. score_og: 1
----------------
evaluating essay...
[6] evaluated. score: 4. score_og: 1
----------------
evaluating essay...
[7] evaluated. score: 4. score_og: 1
----------------
evaluating essay...
[8] evaluated. score: 2. score_og: 2
----------------
evaluating essay...
[9] evaluated. score: 4. score_og: 2
----------------
evaluating essay...
[10] evaluated. score: 4. score_og: 2
----------------
evaluating essay...
[11] evaluated. score: 4. score_og: 2
----------------
evaluating essay...
[12] evaluated. score: 2. score_og: 2
----------------
evaluating essay..

,essay_code,full_text,attribute,label,score_og,score_llm
0,1859-O,"the passage ""in german suburb, life goes on wi...",formality,1,1,4
1,1859-CFO,"The passage ""in german suburb, life goes on wi...",formality,0,1,5
2,19152-O,to me i think we should change the election by...,formality,1,1,3
3,19152-CFO,"To me, I believe that we should alter the elec...",formality,0,1,4
4,8542-O,the challenge of exploring venus\r\n\r\non thi...,formality,1,1,3
...,...,...,...,...,...,...
195,18967-CFO,"candidate, what do you think when I say that w...",formality,0,1,2
196,19652-O,"dear senator of state,\r\n\r\ni think keeping ...",formality,1,1,4
197,19652-CFO,"Dear Senator of State,\n\nI believe that maint...",formality,0,1,5
198,9618-O,"you are smile, but you only 80 percent happy, ...",formality,1,3,4


In [34]:
# save scored counterfactuals to their own folder

os.makedirs("counterfactuals_scored", exist_ok=True)

stance_pro_to_con_scored_df.to_csv("counterfactuals_scored/stance_pro_to_con_scored.csv", index=False)
stance_con_to_pro_scored_df.to_csv("counterfactuals_scored/stance_con_to_pro_scored.csv", index=False)

sentiment_positive_to_negative_scored_df.to_csv("counterfactuals_scored/sentiment_positive_to_negative_scored.csv", index=False)
sentiment_negative_to_positive_scored_df.to_csv("counterfactuals_scored/sentiment_negative_to_positive_scored.csv", index=False)

formality_formal_to_informal_scored_df.to_csv("counterfactuals_scored/formality_formal_to_informal_scored.csv", index=False)
formality_informal_to_formal_scored_df.to_csv("counterfactuals_scored/formality_informal_to_formal_scored.csv", index=False)

In [30]:
# computes qwk for the scored counterfactuals

compute_qwk(stance_pro_to_con_scored_df)
compute_qwk(stance_con_to_pro_scored_df)
compute_qwk(sentiment_positive_to_negative_scored_df)
compute_qwk(sentiment_negative_to_positive_scored_df)
compute_qwk(formality_formal_to_informal_scored_df)
compute_qwk(formality_informal_to_formal_scored_df)

QWK (Originals): 0.3673
QWK (Counterfactuals): 0.3003
QWK (Originals): 0.5573
QWK (Counterfactuals): 0.3524
QWK (Originals): 0.4284
QWK (Counterfactuals): 0.3426
QWK (Originals): 0.5625
QWK (Counterfactuals): 0.2925
QWK (Originals): 0.4385
QWK (Counterfactuals): 0.4490
QWK (Originals): 0.4296
QWK (Counterfactuals): 0.1951


(np.float64(0.429574374725757), np.float64(0.19514482183788295))

In [32]:
# evaluate essays using llama3:8b LLM

stance_pro_to_con_scored_llama_df = evaluate_essays(stance_pro_to_con_df)
stance_con_to_pro_scored_llama_df = evaluate_essays(stance_con_to_pro_df)
sentiment_positive_to_negative_scored_llama_df = evaluate_essays(sentiment_positive_to_negative_df)
sentiment_negative_to_positive_scored_llama_df = evaluate_essays(sentiment_negative_to_positive_df)
formality_formal_to_informal_scored_llama_df = evaluate_essays(formality_formal_to_informal_df)
formality_informal_to_formal_scored_llama_df = evaluate_essays(formality_informal_to_formal_df)

----------------
evaluating essay...
[0] evaluated. score: 2. score_og: 1
----------------
evaluating essay...
[1] evaluated. score: 2. score_og: 1
----------------
evaluating essay...
[2] evaluated. score: 4. score_og: 1
----------------
evaluating essay...
[3] evaluated. score: 4. score_og: 1
----------------
evaluating essay...
[4] evaluated. score: 4. score_og: 1
----------------
evaluating essay...
[5] evaluated. score: 4. score_og: 1
----------------
evaluating essay...
[6] evaluated. score: 4. score_og: 1
----------------
evaluating essay...
[7] evaluated. score: 4. score_og: 1
----------------
evaluating essay...
[8] evaluated. score: 2. score_og: 2
----------------
evaluating essay...
[9] evaluated. score: 4. score_og: 2
----------------
evaluating essay...
[10] evaluated. score: 4. score_og: 2
----------------
evaluating essay...
[11] evaluated. score: 4. score_og: 2
----------------
evaluating essay...
[12] evaluated. score: 2. score_og: 2
----------------
evaluating essay..

In [33]:
# computes qwk of the scored with llama3 counterfactuals

compute_qwk(stance_pro_to_con_scored_llama_df)
compute_qwk(stance_con_to_pro_scored_llama_df)
compute_qwk(sentiment_positive_to_negative_scored_llama_df)
compute_qwk(sentiment_negative_to_positive_scored_llama_df)
compute_qwk(formality_formal_to_informal_scored_llama_df)
compute_qwk(formality_informal_to_formal_scored_llama_df)

QWK (Originals): 0.0851
QWK (Counterfactuals): 0.1385
QWK (Originals): 0.0940
QWK (Counterfactuals): -0.0174
QWK (Originals): 0.0308
QWK (Counterfactuals): 0.0557
QWK (Originals): 0.1043
QWK (Counterfactuals): 0.0276
QWK (Originals): 0.0874
QWK (Counterfactuals): 0.0471
QWK (Originals): 0.0652
QWK (Counterfactuals): 0.0071


(np.float64(0.06518185533306098), np.float64(0.007070076939072356))